In [ ]:
pip install recommenders

     |████████████████████████████████| 314 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 27.6 MB/s 
     |████████████████████████████████| 1.5 MB 20.5 MB/s 
     |████████████████████████████████| 310 kB 53.2 MB/s 
     |████████████████████████████████| 11.8 MB 32.7 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 12.4 MB 46.4 MB/s 
     |████████████████████████████████| 636 kB 37.5 MB/s 
     |████████████████████████████████| 93 kB 2.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 61 kB 6.6 MB/s 
     |████████████████████████████████| 749 kB 73.2 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 61 kB 472 kB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705341 sha256=ec2dd0f78e07421141b03271c96b312840b306d

In [ ]:
pip install papermill

     |████████████████████████████████| 156 kB 6.7 MB/s 
     |████████████████████████████████| 129 kB 39.5 MB/s 
     |████████████████████████████████| 843 kB 38.3 MB/s 
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


In [ ]:
pip install tensorflow==1.15.2

     |████████████████████████████████| 110.5 MB 33 kB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 3.8 MB 16.3 MB/s 
     |████████████████████████████████| 503 kB 40.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2c34f2a6e32e362a13e2a75b42b4bb0eae7b1d5ba7c616b5bc5a6fc611b142d1
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:

import sys
import os
import shutil
import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Pandas version: 1.1.5
Tensorflow version: 1.15.2


## Data

In [ ]:
cmt=pd.read_csv('/content/drive/MyDrive/RS/DATA_vfo/DATASET/Cleaned/COMMENT_cleaned.csv')

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/RS/DATA_vfo/DATASET/Test/test.csv')
test_df=test_df.drop(columns='Unnamed: 0')

In [ ]:
cmt

,id_post,id_author_cmt,url_author_cmt,content_cmt,datePublished_cmt,content_cmt_cleaned
0,0,179483,/m/0406.179483/,\n\nhoangdacviet:\n\n\n\n\n\n\n\nBy SvTre.com ...,2021-06-25T12:03:23+0700,hoangdacviet by svtre com llc màn đẹp thế_bác
1,0,179483,/m/0406.179483/,\n\nguongtp:\n\n\n\n\t\t\tBạn dùng máy cấu hìn...,2021-06-25T12:05:06+0700,guongtp máy cấu_hình_như cấu_hình máy core i3 ...
2,0,466251,/m/clone-vui-tinh.466251/,"Ai cũng Win10, có mình tui Win7",2021-06-25T17:42:27+0700,win10 tui win7
3,0,468305,/m/thien1905.468305/,\n\n\n\n\nmình có cái bàn làm việc thật rộng.....,2021-11-01T09:10:10+0700,bàn làm_việc rộng xong mình ôm laptop giường
4,1,392333,/m/lammori.392333/,\n\n0406:\n\n\n\n\t\t\tVào app My viettel xem ...,2020-09-25T10:56:51+0700,app my viettel làm_sao
...,...,...,...,...,...,...
98863,21817,4,/m/t.4/,Ðề: Cách bỏ dấu trong bài viết\n\nbạn chọn bản...,2011-04-03T12:48:02+0700,ðề dấu viết bảng mã unicode trong vietkey
98864,21817,7384,/m/callofduty-1982.7384/,Ðề: Cách bỏ dấu trong bài viết\n\nđược rồi mìn...,2011-04-03T19:57:39+0700,ðề dấu viết unikey ngố cam ơn tất_cả
98865,21817,2,/m/le-minh.2/,Ðề: Cách bỏ dấu trong bài viết\n\nVì mình đã b...,2011-04-02T21:05:29+0700,ðề dấu viết bỏ bỏ dấu hầu_như máy unikey vietk...
98866,21817,7384,/m/callofduty-1982.7384/,Ðề: Cách bỏ dấu trong bài viết\n\nvẫn không đư...,2011-04-03T12:21:07+0700,ðề dấu viết cài vietkey gõ trong microsoft off...


In [ ]:
timestamp=[]
time=pd.to_datetime(cmt['datePublished_cmt'])
for curr_dt in time:
  timestamp.append(int(round(curr_dt.timestamp())))


In [ ]:
df1=pd.DataFrame()

In [ ]:
df1['userID']=cmt['id_author_cmt']
df1['itemID']=cmt['id_post']
df1['rating']=np.ones(len(timestamp))
df1['timestamp']=timestamp

In [ ]:
df1.drop_duplicates(subset=['userID', 'itemID'], inplace=True)

In [ ]:
df1

,userID,itemID,rating,timestamp
0,179483,0,1.0,1624597403
2,466251,0,1.0,1624617747
3,468305,0,1.0,1635732610
4,392333,1,1.0,1601006211
5,462067,1,1.0,1601883263
...,...,...,...,...
98860,2,21816,1.0,1303220358
98863,4,21817,1.0,1301809682
98864,7384,21817,1.0,1301835459
98865,2,21817,1.0,1301753129


In [ ]:
delete=test_df[test_df['label']==1][['id_user','id_post']]

In [ ]:
delete.columns=['userID', 'itemID']

In [ ]:
index_train=df1[['userID','itemID']]

In [ ]:
df = pd.merge(df1, delete, on=['userID','itemID'], how='left', indicator='Exist')
df.drop(columns=['rating',	'timestamp'], inplace=True, axis=1)
df1['Exist'] = np.where(df.Exist == 'both', True, False)

In [ ]:
#drop sample in test
df1=df1[df1['Exist']==False]
df1

,userID,itemID,rating,timestamp,Exist
0,179483,0,1.0,1624597403,False
2,466251,0,1.0,1624617747,False
3,468305,0,1.0,1635732610,False
4,392333,1,1.0,1601006211,False
5,462067,1,1.0,1601883263,False
...,...,...,...,...,...
98860,2,21816,1.0,1303220358,False
98863,4,21817,1.0,1301809682,False
98864,7384,21817,1.0,1301835459,False
98865,2,21817,1.0,1301753129,False


## Model

In [ ]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 128

SEED = DEFAULT_SEED  # Set None for non-deterministic results

In [ ]:
train, test = python_chrono_split(df1, 0.999999)

In [ ]:
data = NCFDataset(train=train, test=test, seed=SEED)

In [ ]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=8,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [ ]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 913.819953299 seconds for training.


In [ ]:
list_user_test=test_df['id_user'].unique()

In [ ]:
k = 10
ndcgs = []
hit_ratio = []

for id in list_user_test:
  output=[]
  temp=test_df[test_df['id_user']==id]
  list_item=temp['id_post'].unique()
  for item in list_item:
    try:
      output.append(model.predict(id, item))
    except:
      pass
  output = np.squeeze(output)
  rank = sum(output >= output[0])
  if rank <= k:
      ndcgs.append(1 / np.log(rank + 1))
      hit_ratio.append(1)
  else:
      ndcgs.append(0)
      hit_ratio.append(0)

eval_ndcg = np.mean(ndcgs)
eval_hr = np.mean(hit_ratio)

print("HR:\t%f" % eval_hr)
print("NDCG:\t%f" % eval_ndcg)


HR:	0.250000
NDCG:	0.209972
